In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
imgsize = (224,224)
trainfolder= './Train/'
# trainfolder="/kaggle/input/nn22-weather-analysis-using-image-recognition/Dataset/Train/"
batch_size = 64
aug_factor = 1
sub_dir=['dew','fogsmog','frost','glaze','hail',
         'lightning','rain','rainbow','rime','sandstorm','snow']
TRAIN_DIR="/kaggle/input/nn22-weather-analysis-using-image-recognition/Dataset/Train/"


In [2]:
import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
import tensorflow as tf
import tqdm

In [3]:
import os

sub_dir_aug = []
sub_dir_sizes = []
max_class = 0
for sub_file in sub_dir :
    sub=TRAIN_DIR+sub_file
    size=len(os.listdir(sub))
    sub_dir_sizes.append(size)
    if size > max_class :
        max_class = size
print (max_class)

for size in sub_dir_sizes :
    sub_dir_aug.append(round(max_class*aug_factor/size))
sub_dir_aug

1059


[2, 1, 3, 2, 2, 4, 3, 8, 1, 2, 2]

In [4]:
import os
def create_output_path() :
    output_path = './'
    dire=''
    if os.path.isdir(output_path+"Train")==False :
        os.mkdir(output_path+"Train")
    output_path = './Train/'
    for i in range(len(sub_dir)) :
        dire=output_path+sub_dir[i]
        if os.path.isdir(dire)==False :
            os.mkdir(output_path+sub_dir[i])
    return output_path

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import numpy as np
import cv2
aug = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
#     vertical_flip=True,
    fill_mode="nearest")

def single_data_over_sampling(imagename , total , saved_dir,aug=aug) :
    image = load_img(imagename)
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)

    imageGen = aug.flow(image, batch_size=1, save_to_dir=saved_dir,
        save_prefix="image", save_format="jpg")
    # loop over examples from our image data augmentation generator
    i=0
    for image in imageGen:
        # increment our counter
        i += 1
        # if we have reached the specified number of examples, break
        # from the loop
        if i == total:
            break

In [6]:
# sub_dir_aug
# TRAIN_DIR
# sub_dir
import os
output_path = create_output_path()
def data_over_sampling():
    for i in range(len(sub_dir)) :
        sub = TRAIN_DIR+sub_dir[i]
        images = os.listdir(sub)
        for image in images :
            single_data_over_sampling(sub+'/'+image , sub_dir_aug[i] , output_path+sub_dir[i])

In [7]:
data_over_sampling()

In [8]:
for sub_file in sub_dir :
    sub=trainfolder+sub_file
    size=len(os.listdir(sub))
    print(size)

1052
721
956
1029
849
1057
1109
1001
1006
1042
987


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

TRAIN_DATAGEN = ImageDataGenerator(rescale = 1./255.,
                                  rotation_range = 40,
                                  width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   validation_split=0.2,
                                    vertical_flip=True,
                                   horizontal_flip = True,
                                    fill_mode='nearest',
                                   
                                  )
VALID_DATAGEN = ImageDataGenerator(rescale = 1./255.,
                                  rotation_range = 40,
                                  width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   validation_split=0.2,
                                    vertical_flip=True,
                                    horizontal_flip = True,
                                   fill_mode='nearest',
                                   
                                  )


TRAIN_GENERATOR = TRAIN_DATAGEN.flow_from_directory(directory = trainfolder,
                                                    batch_size = batch_size,
                                                    target_size = imgsize,
                                                    subset = 'training',
                                                    color_mode='rgb',
                                                    seed = 123)

VALIDATION_GENERATOR = VALID_DATAGEN.flow_from_directory(directory = trainfolder,
                                                        batch_size = batch_size, 
                                                        target_size = imgsize,
                                                        subset = 'validation',
                                                        color_mode='rgb',
                                                        seed = 123)

Found 8652 images belonging to 11 classes.
Found 2157 images belonging to 11 classes.


In [10]:
from collections import Counter
counter = Counter(TRAIN_GENERATOR.classes)                          
max_val = float(max(counter.values()))       
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}                     

In [11]:
# # from tflearn.layers.conv import conv_2d, max_pool_2d
# # from tflearn.layers.core import input_data, dropout, fully_connected
# # from tflearn.layers.estimator import regression
# # from sklearn.model_selection import train_test_split
# # X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=1,shuffle=True)

# from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Reshape , Conv2D,MaxPooling2D,Dropout,UpSampling2D,concatenate,BatchNormalization
# from tensorflow.keras.models import Model
# from tensorflow.keras import optimizers, losses, metrics,regularizers

# kernel_initializer = 'he_normal'
# def model(shape):
    
# #     conv_input = input_data(shape=[None, 256, 256, 3], name='input')
#     inputs = Input(shape)
#     conv1 = Conv2D( 32, 5, activation='relu',padding = 'same', kernel_initializer = kernel_initializer)(inputs)
#     pool1 = MaxPooling2D(5)(conv1)

#     conv2 = Conv2D( 64, 5, activation='relu',padding = 'same', kernel_initializer = kernel_initializer)(pool1)
#     pool2 = MaxPooling2D( 5)(conv2)

#     conv3 = Conv2D( 128, 5, activation='relu',padding = 'same', kernel_initializer = kernel_initializer)(pool2)
#     pool3 = MaxPooling2D( 5)(conv3)

#     conv4 = Conv2D( 64, 5, activation='relu',padding = 'same', kernel_initializer = kernel_initializer)(pool3)
#     pool4 = MaxPooling2D(5,padding = 'same')(conv4)

#     conv5 = Conv2D( 32, 5, activation='relu',padding = 'same', kernel_initializer = kernel_initializer)(pool4)
#     pool5 = MaxPooling2D(5,padding = 'same')(conv5)
    
#     flatten = Flatten()(pool5)
    
#     fully_layer = Dense( 1024, activation='relu', kernel_initializer = kernel_initializer)(flatten)
#     fully_layer = Dropout( 0.5)(fully_layer)

#     cnn_layers = Dense( 11, activation='softmax')(fully_layer)

# #     cnn_layers = regression(cnn_layers, optimizer='adam', learning_rate=0.001, loss='categorical_crossentropy', name='targets')
# #     model = tflearn.DNN(cnn_layers, tensorboard_dir='log', tensorboard_verbose=3)
#     return  Model(inputs, cnn_layers)

In [12]:
def covidmodel(shape) :
    # Creating a Sequential Model and adding the layers
    modelXCOVID1 = Sequential()
    modelXCOVID1.add(Conv2D(filters=32, input_shape=shape, kernel_size=(11,11), strides=(4,4), padding='valid'))
    modelXCOVID1.add(Activation('relu'))
    #Add bTCH NORMALIZATION LAYER 
    #BatchNormalization(axis=1)
    modelXCOVID1.add(MaxPooling2D(pool_size=(2,2)))
    modelXCOVID1.add(Conv2D(filters=64, kernel_size=(3,3)))
    modelXCOVID1.add(Activation('relu'))
    #Add bTCH NORMALIZATION LAYER 
    #BatchNormalization(axis=1)
    modelXCOVID1.add(MaxPooling2D(pool_size=(2, 2)))
    modelXCOVID1.add(Conv2D(128, kernel_size=(3,3) ))
    modelXCOVID1.add(Activation('relu'))
    #Add bTCH NORMALIZATION LAYER 
    #BatchNormalization(axis=1)
    modelXCOVID1.add(MaxPooling2D(pool_size=(2, 2)))
    modelXCOVID1.add(Flatten())
    # 1st Fully Connected Layer
    modelXCOVID1.add(Dense(512))
    modelXCOVID1.add(Activation('relu'))
    # Add Dropout to prevent overfitting
    modelXCOVID1.add(Dropout(0.4))
    # 2rd Fully Connected Layer
    modelXCOVID1.add(Dense(64))
    modelXCOVID1.add(Activation('relu'))
    # Add Dropout to prevent overfitting
    modelXCOVID1.add(Dropout(0.4))
    # Passing it to a Fully Connected layer

    # Output Layer
    modelXCOVID1.add(Dense(11))
    modelXCOVID1.add(Activation('softmax'))
    
    return modelXCOVID1


In [13]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Reshape , Conv2D,MaxPooling2D,Dropout,UpSampling2D,concatenate,BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers, losses, metrics,regularizers
def vgg(shape):
    model = Sequential()
    model.add(Conv2D(input_shape=shape,filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Flatten())
#     model.add(Dense(units=4096,activation="relu"))
#     model.add(Dense(units=1024,activation="relu"))
    model.add(Dense(units=11, activation="softmax"))
    
    return model

In [14]:
from keras import backend as K
import keras
from keras.models import Sequential, Model,load_model
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from keras.preprocessing import image
from keras.initializers import glorot_uniform

In [15]:
def identity_block(X, f, filters, stage, block):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters

    X_shortcut = X
   
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X = Add()([X, X_shortcut])# SKIP Connection
    X = Activation('relu')(X)

    return X
def convolutional_block(X, f, filters, stage, block, s=2):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X
def ResNet50(input_shape=(224, 224, 3)):

    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X_input)

    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')


    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    X = X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)
    
    X = Flatten()(X)
#     X = Dense( 2048, activation='relu', kernel_initializer=glorot_uniform(seed=0))(X)
#     X = Dropout( 0.5)(X)
    X = Dense( 1024, activation='relu', kernel_initializer=glorot_uniform(seed=0))(X)
    X = Dropout( 0.5)(X)
    
    X = Dense( 11, activation='softmax')(X)

    
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model

In [16]:
import keras
from keras.layers.core import Layer
import keras.backend as K
import tensorflow as tf
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D,  \
    Dropout, Dense, Input, concatenate,      \
    GlobalAveragePooling2D, AveragePooling2D,\
    Flatten

import cv2 
import numpy as np 
from keras.datasets import cifar10 
from keras import backend as K 
from keras.utils import np_utils

import math 
# from keras.optimizers import SGD 
# from keras.callbacks import LearningRateScheduler
def inception_module(x,
                     filters_1x1,
                     filters_3x3_reduce,
                     filters_3x3,
                     filters_5x5_reduce,
                     filters_5x5,
                     filters_pool_proj,
                     name=None):
    
    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    
    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_3x3)

    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_5x5)

    pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(pool_proj)

    output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)
    
    return output
kernel_init = keras.initializers.glorot_uniform()
bias_init = keras.initializers.Constant(value=0.2)
def inception(shape):
    input_layer = Input(shape=shape)

    x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', name='conv_1_7x7/2', kernel_initializer=kernel_init, bias_initializer=bias_init)(input_layer)
    x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_1_3x3/2')(x)
    x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu', name='conv_2a_3x3/1')(x)
    x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', name='conv_2b_3x3/1')(x)
    x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_2_3x3/2')(x)

    x = inception_module(x,
                         filters_1x1=64,
                         filters_3x3_reduce=96,
                         filters_3x3=128,
                         filters_5x5_reduce=16,
                         filters_5x5=32,
                         filters_pool_proj=32,
                         name='inception_3a')

    x = inception_module(x,
                         filters_1x1=128,
                         filters_3x3_reduce=128,
                         filters_3x3=192,
                         filters_5x5_reduce=32,
                         filters_5x5=96,
                         filters_pool_proj=64,
                         name='inception_3b')

    x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_3_3x3/2')(x)

    x = inception_module(x,
                         filters_1x1=192,
                         filters_3x3_reduce=96,
                         filters_3x3=208,
                         filters_5x5_reduce=16,
                         filters_5x5=48,
                         filters_pool_proj=64,
                         name='inception_4a')


    x1 = AveragePooling2D((5, 5), strides=3)(x)
    x1 = Conv2D(128, (1, 1), padding='same', activation='relu')(x1)
    x1 = Flatten()(x1)
    x1 = Dense(1024, activation='relu')(x1)
    x1 = Dropout(0.7)(x1)
    x1 = Dense(11, activation='softmax', name='auxilliary_output_1')(x1)

    x = inception_module(x,
                         filters_1x1=160,
                         filters_3x3_reduce=112,
                         filters_3x3=224,
                         filters_5x5_reduce=24,
                         filters_5x5=64,
                         filters_pool_proj=64,
                         name='inception_4b')

    x = inception_module(x,
                         filters_1x1=128,
                         filters_3x3_reduce=128,
                         filters_3x3=256,
                         filters_5x5_reduce=24,
                         filters_5x5=64,
                         filters_pool_proj=64,
                         name='inception_4c')

    x = inception_module(x,
                         filters_1x1=112,
                         filters_3x3_reduce=144,
                         filters_3x3=288,
                         filters_5x5_reduce=32,
                         filters_5x5=64,
                         filters_pool_proj=64,
                         name='inception_4d')


    x2 = AveragePooling2D((5, 5), strides=3)(x)
    x2 = Conv2D(128, (1, 1), padding='same', activation='relu')(x2)
    x2 = Flatten()(x2)
    x2 = Dense(1024, activation='relu')(x2)
    x2 = Dropout(0.7)(x2)
    x2 = Dense(11, activation='softmax', name='auxilliary_output_2')(x2)

    x = inception_module(x,
                         filters_1x1=256,
                         filters_3x3_reduce=160,
                         filters_3x3=320,
                         filters_5x5_reduce=32,
                         filters_5x5=128,
                         filters_pool_proj=128,
                         name='inception_4e')

    x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_4_3x3/2')(x)

    x = inception_module(x,
                         filters_1x1=256,
                         filters_3x3_reduce=160,
                         filters_3x3=320,
                         filters_5x5_reduce=32,
                         filters_5x5=128,
                         filters_pool_proj=128,
                         name='inception_5a')

    x = inception_module(x,
                         filters_1x1=384,
                         filters_3x3_reduce=192,
                         filters_3x3=384,
                         filters_5x5_reduce=48,
                         filters_5x5=128,
                         filters_pool_proj=128,
                         name='inception_5b')

    x = GlobalAveragePooling2D(name='avg_pool_5_3x3/1')(x)
    
#     x = Dense( 2048, activation='relu', kernel_initializer=glorot_uniform(seed=0))(x)
#     x = Dropout( 0.5)(x)
#     x = Dense( 1024, activation='relu', kernel_initializer=glorot_uniform(seed=0))(x)
#     x = Dropout( 0.5)(x)

    x = Dropout(0.5)(x)

    x = Dense(11, activation='softmax', name='output')(x)
    
    return Model(input_layer, [x, x1, x2], name='inception_v1')
    

In [17]:
ince=False
# model = model((224, 224, 3))
# model = vgg((224, 224, 3))
# model = covidmodel((224, 224, 3))
model = ResNet50((224, 224, 3))
# model = inception((224, 224, 3))
# ince=True
model.summary()


2022-05-24 22:54:04.934134: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-24 22:54:05.057979: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-24 22:54:05.058798: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-24 22:54:05.060140: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [18]:
def focal_loss(gamma=2., alpha=4.):

    gamma = float(gamma)
    alpha = float(alpha)

    def focal_loss_fixed(y_true, y_pred):
        """Focal loss for multi-classification
        FL(p_t)=-alpha(1-p_t)^{gamma}ln(p_t)
        Notice: y_pred is probability after softmax
        gradient is d(Fl)/d(p_t) not d(Fl)/d(x) as described in paper
        d(Fl)/d(p_t) * [p_t(1-p_t)] = d(Fl)/d(x)
        Focal Loss for Dense Object Detection
        https://arxiv.org/abs/1708.02002

        Arguments:
            y_true {tensor} -- ground truth labels, shape of [batch_size, num_cls]
            y_pred {tensor} -- model's output, shape of [batch_size, num_cls]

        Keyword Arguments:
            gamma {float} -- (default: {2.0})
            alpha {float} -- (default: {4.0})

        Returns:
            [tensor] -- loss.
        """
        epsilon = 1.e-9
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)

        model_out = tf.add(y_pred, epsilon)
        ce = tf.multiply(y_true, -tf.math.log(model_out))
        weight = tf.multiply(y_true, tf.pow(tf.subtract(1., model_out), gamma))
        fl = tf.multiply(alpha, tf.multiply(weight, ce))
        reduced_fl = tf.reduce_max(fl, axis=1)
        return tf.reduce_mean(reduced_fl)
    return focal_loss_fixed

In [19]:
epochs = 100
# lr = 0.0004
# batch_size = 64
optimizer = optimizers.Adam()
# optimizer = optimizers.
# loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = ['accuracy']
model.compile(optimizer=optimizer,loss='categorical_crossentropy' , metrics=metric)#lloss='categorical_crossentropy' ,loss=focal_loss(alpha=11),

In [20]:
import math
TRAIN_GENERATOR.samples//batch_size

135

In [21]:
from keras.callbacks import EarlyStopping
his = model.fit(TRAIN_GENERATOR,validation_data=VALIDATION_GENERATOR,epochs=epochs,batch_size=batch_size,class_weight=class_weights,
               steps_per_epoch= math.ceil(TRAIN_GENERATOR.samples//batch_size),validation_steps=math.ceil(VALIDATION_GENERATOR.samples//batch_size),callbacks = [EarlyStopping(monitor='val_acc', patience=2)])

# his = model.fit(TRAIN_GENERATOR,epochs=epochs,batch_size=batch_size,class_weight=class_weights,
#                steps_per_epoch= math.ceil(TRAIN_GENERATOR.samples//batch_size),callbacks = [EarlyStopping(monitor='val_acc', patience=2)])

model.save('model.h5')

2022-05-24 22:54:11.478972: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100


2022-05-24 22:54:18.062998: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


135/135 [==============================] - 181s 1s/step - loss: 8.6892 - accuracy: 0.1410 - val_loss: 2.4554 - val_accuracy: 0.0914
Epoch 2/100
135/135 [==============================] - 169s 1s/step - loss: 2.2458 - accuracy: 0.2923 - val_loss: 2.5114 - val_accuracy: 0.2159
Epoch 3/100
135/135 [==============================] - 166s 1s/step - loss: 1.8855 - accuracy: 0.4099 - val_loss: 2.7273 - val_accuracy: 0.2448
Epoch 4/100
135/135 [==============================] - 170s 1s/step - loss: 1.7052 - accuracy: 0.4765 - val_loss: 1.8638 - val_accuracy: 0.3726
Epoch 5/100
135/135 [==============================] - 168s 1s/step - loss: 1.6117 - accuracy: 0.5052 - val_loss: 1.6128 - val_accuracy: 0.4593
Epoch 6/100
135/135 [==============================] - 167s 1s/step - loss: 1.5717 - accuracy: 0.5227 - val_loss: 1.2462 - val_accuracy: 0.5511
Epoch 7/100
135/135 [==============================] - 167s 1s/step - loss: 1.5074 - accuracy: 0.5409 - val_loss: 1.4669 - val_accuracy: 0.5232
Epoc

/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [22]:
TEST_DATAGEN = ImageDataGenerator(rescale = 1.0/255.)
testfolder= '/kaggle/input/nn22-weather-analysis-using-image-recognition/Dataset/'
TEST_GENERATOR = TEST_DATAGEN.flow_from_directory(directory = testfolder,
                                                    target_size = imgsize,
                                                   shuffle=False,
                                                  color_mode = 'rgb',
                                                  class_mode='categorical',
                                                   classes=['Test']
                                                  )
TEST_GENERATOR.reset()

Found 1300 images belonging to 1 classes.


In [23]:
prediction = model.predict_generator(generator=TEST_GENERATOR,steps=len(TEST_GENERATOR), verbose=1)
prediction

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


41/41 [==============================] - 16s 353ms/step


array([[6.1321600e-08, 6.0294719e-06, 4.7654743e-05, ..., 8.2870081e-02,
        1.2148679e-08, 8.4554005e-01],
       [3.9248559e-07, 4.5701286e-06, 9.4298157e-05, ..., 6.5275833e-02,
        7.4796791e-09, 6.7587721e-01],
       [3.0601569e-04, 9.6511483e-01, 2.5839239e-04, ..., 5.4695626e-04,
        1.9027386e-02, 9.8634167e-03],
       ...,
       [4.4423019e-04, 3.3994771e-03, 1.3978196e-04, ..., 1.4270784e-04,
        1.4166746e-03, 5.1283359e-04],
       [2.7448614e-05, 3.9911969e-03, 2.8667433e-04, ..., 4.4872243e-02,
        5.6613067e-06, 6.8845296e-01],
       [2.1203633e-02, 8.5736701e-04, 2.1479634e-04, ..., 1.1856114e-03,
        1.0264212e-03, 8.5993178e-02]], dtype=float32)

In [24]:
import numpy as np
if ince==True :    
    pred = np.argmax(prediction[0],axis=1)
else :
    pred = np.argmax(prediction,axis=1)
pred

array([10, 10,  1, ...,  7, 10,  4])

In [25]:
pred.shape

(1300,)

In [26]:
def imagenamsfix(imgnames) :

    imgnames = np.char.split(imgnames,sep='/')
    names = []
    for i in imgnames :
        names.append(i[-1])
    return np.array(names)

In [27]:
import pandas as pd
filenames=TEST_GENERATOR.filenames
sub = pd.DataFrame({"image_name":imagenamsfix(filenames),"label":pred})


In [30]:
sub.to_csv('submission2.csv' , index=False)

In [29]:
sub

,image_name,label
0,test_1.jpg,10
1,test_10.jpg,10
2,test_100.jpg,1
3,test_1000.jpg,7
4,test_1001.jpg,4
...,...,...
1295,test_995.jpg,8
1296,test_996.jpg,4
1297,test_997.jpg,7
1298,test_998.jpg,10
